**Proposition 1)** Customers who bought a road bike in 2013 but did not buy any bike in 2014

- creates a Common Table Expression "BikeSales" to simplify filtering and reuse bike order data.
- Retrieves customers who bought "Road Bikes" in 2013
- uses the **EXCEPT** operator to exlcude customers who bought any bike in 2014.
- Returns customers who stopped purchasing bikes after 2013.

Why Special: Helps marketing teams identify customres who stopped purchasing a specific product line, where you can try to win these customers back.

In [ ]:
WITH BikeSales AS (
  SELECT soh.CustomerID, soh.OrderDate, psc.Name AS SubCat
  FROM Sales.SalesOrderHeader soh
  JOIN Sales.SalesOrderDetail sod ON sod.SalesOrderID = soh.SalesOrderID
  JOIN Production.Product p ON p.ProductID = sod.ProductID
  JOIN Production.ProductSubcategory psc ON psc.ProductSubcategoryID = p.ProductSubcategoryID
  JOIN Production.ProductCategory pc ON pc.ProductCategoryID = psc.ProductCategoryID
  WHERE pc.Name = N'Bikes'
)
SELECT DISTINCT CustomerID
FROM BikeSales
WHERE SubCat = N'Road Bikes'
  AND OrderDate >= '20130101' AND OrderDate < '20140101'

EXCEPT

SELECT DISTINCT CustomerID
FROM BikeSales
WHERE OrderDate >= '20140101' AND OrderDate < '20150101';

**Proposition 2)** Products sold in North America but not Europe

- The inner part (**EXCEPT**) finds ProductIDs sold in North America but not in Europe.
- The outer query joins that result to the Production.product table, pulling in each item's name.
- DISTINCT ensures you dont get duplicate rows.

Why Special: Useful for international sales analysis to find regional product gaps or tailor distribution for other countries.

In [ ]:
SELECT DISTINCT p.ProductID, p.Name AS ProductName
FROM Production.Product AS p
WHERE p.ProductID IN (

    
        SELECT DISTINCT sod.ProductID
        FROM Sales.SalesOrderHeader AS soh
        JOIN Sales.SalesOrderDetail AS sod
            ON soh.SalesOrderID = sod.SalesOrderID
        JOIN Sales.SalesTerritory AS st
            ON soh.TerritoryID = st.TerritoryID
        WHERE st.CountryRegionCode IN ('US', 'CA', 'MX')
    

    EXCEPT
    
        SELECT DISTINCT sod.ProductID
        FROM Sales.SalesOrderHeader AS soh
        JOIN Sales.SalesOrderDetail AS sod
            ON soh.SalesOrderID = sod.SalesOrderID
        JOIN Sales.SalesTerritory AS st
            ON soh.TerritoryID = st.TerritoryID
        WHERE st.CountryRegionCode IN ('GB', 'FR', 'DE')
    
)
ORDER BY p.ProductID;


**Proposition 3)** Employees who are also salespeople with sales over $1,000,000

- Uses **INTERSECT** operator to find people who appear in both HumanResources.Employee and Sales.Salesperson
- Joins the matching employees to Sales.SalesOrderHeader to calculate their total sales.
- Groups results by salesperson and filters where SUM(TotalDue)\>1,000,000

Why Special: Enables HR and finance to recognize high-performing employees who directly drive major sales to the business.

In [ ]:
WITH EmployeeSalespeople AS (
    SELECT BusinessEntityID
    FROM HumanResources.Employee

    INTERSECT

    SELECT BusinessEntityID
    FROM Sales.SalesPerson
)
SELECT
    Person.BusinessEntityID AS SalespersonID,
    Person.FirstName + ' ' + Person.LastName AS SalespersonName,
    SUM(SalesOrderHeader.TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader AS SalesOrderHeader
JOIN EmployeeSalespeople AS EmployeeSalespeople
  ON SalesOrderHeader.SalesPersonID = EmployeeSalespeople.BusinessEntityID
JOIN Person.Person AS Person
  ON Person.BusinessEntityID = EmployeeSalespeople.BusinessEntityID
GROUP BY
    Person.BusinessEntityID,
    Person.FirstName,
    Person.LastName
HAVING SUM(SalesOrderHeader.TotalDue) > 1000000
ORDER BY TotalSales DESC;

**Proposition 4)** A combined list of territories assigned to salespersons and territories where stores are located.

- Uses the UNION operator to combine territories form both Sales.SalesPerson and Sales.Customer.
- Joins the results with Sales.SalesTerritory to display the territory name, region, and group.
- Ensures only distinct, non-repeating territories are shown.

Why Special: Assists operations coordinators/managers to map sales coverage across territories, to ensure no overlap or missed regions.

In [ ]:
SELECT DISTINCT
    Territory.TerritoryID,
    Territory.Name AS TerritoryName,
    Territory.CountryRegionCode,
    Territory.[Group] AS TerritoryGroup
FROM Sales.SalesTerritory AS Territory
WHERE Territory.TerritoryID IN (
    SELECT TerritoryID
    FROM Sales.SalesPerson
    WHERE TerritoryID IS NOT NULL

    UNION

    SELECT TerritoryID
    FROM Sales.Customer
    WHERE StoreID IS NOT NULL
      AND TerritoryID IS NOT NULL
)
ORDER BY TerritoryName;

**Proposition 5)** Salespeople who sold bikes and Components

- Creates a CTE named SalesLines that joins sales and product tables to include each product's category.
- Filters that CTE twice to get salespeople who sold Bikes and those who sold Components.
- Uses an INTERSECT operator to return only salespeople found in both sets.

Why Special : Helps sales manageemnt identify cross-category sellers who can handle diverse products and sell them both.

In [ ]:
WITH SalesLines AS (
    SELECT
        SalesOrderHeader.SalesPersonID,
        SalesOrderHeader.SalesOrderID,
        ProductCategory.Name AS ProductCategory
    FROM Sales.SalesOrderHeader AS SalesOrderHeader
    JOIN Sales.SalesOrderDetail AS SalesOrderDetail
        ON SalesOrderHeader.SalesOrderID = SalesOrderDetail.SalesOrderID
    JOIN Production.Product AS Product
        ON Product.ProductID = SalesOrderDetail.ProductID
    JOIN Production.ProductSubcategory AS ProductSubcategory
        ON ProductSubcategory.ProductSubcategoryID = Product.ProductSubcategoryID
    JOIN Production.ProductCategory AS ProductCategory
        ON ProductCategory.ProductCategoryID = ProductSubcategory.ProductCategoryID
    WHERE SalesOrderHeader.SalesPersonID IS NOT NULL
)
SELECT DISTINCT SalesPersonID
FROM SalesLines
WHERE ProductCategory = N'Bikes'

INTERSECT

SELECT DISTINCT SalesPersonID
FROM SalesLines
WHERE ProductCategory = N'Components';


**Proposition 6)** Vendors who supply parts for products sold both online and in stores.

- Build the set of vendor IDs whose products appear on online orders, and the in-store orders.
- Use INTERSECT to keep only vendors present in both sets

Why Special: Useful for finding out which vendors are supporting both online and in-store sales.

In [ ]:
SELECT DISTINCT Vendor.BusinessEntityID, Vendor.Name AS VendorName
FROM Purchasing.Vendor AS Vendor
WHERE Vendor.BusinessEntityID IN (

        SELECT DISTINCT ProductVendor.BusinessEntityID
        FROM Purchasing.ProductVendor AS ProductVendor
        WHERE ProductVendor.ProductID IN (
            SELECT DISTINCT SalesOrderDetail.ProductID
            FROM Sales.SalesOrderDetail AS SalesOrderDetail
            JOIN Sales.SalesOrderHeader AS SalesOrderHeader
              ON SalesOrderHeader.SalesOrderID = SalesOrderDetail.SalesOrderID
            WHERE SalesOrderHeader.OnlineOrderFlag = 1
        )

    INTERSECT

        SELECT DISTINCT ProductVendor.BusinessEntityID
        FROM Purchasing.ProductVendor AS ProductVendor
        WHERE ProductVendor.ProductID IN (
            SELECT DISTINCT SalesOrderDetail.ProductID
            FROM Sales.SalesOrderDetail AS SalesOrderDetail
            JOIN Sales.SalesOrderHeader AS SalesOrderHeader
              ON SalesOrderHeader.SalesOrderID = SalesOrderDetail.SalesOrderID
            WHERE SalesOrderHeader.OnlineOrderFlag = 0
        )
)
ORDER BY VendorName;

**Proposition 7)** Customers who have purchased products on special offers

- The first SELECT gets all customres from Sales.Customer
- The second query finds customers tied to orders containing any product where SpecialOfferID !=1 (meaning part of a promo)
- The INTERSECT operator keeps only those customers who appears in both sets

Why Special: Enables marketing teams to find customers who respond to special offers, maybe target them more

In [ ]:
SELECT Customer.CustomerID
FROM Sales.Customer AS Customer

INTERSECT

SELECT DISTINCT SalesOrderHeader.CustomerID
FROM Sales.SalesOrderHeader AS SalesOrderHeader
JOIN Sales.SalesOrderDetail AS SalesOrderDetail
    ON SalesOrderHeader.SalesOrderID = SalesOrderDetail.SalesOrderID
WHERE SalesOrderDetail.SpecialOfferID != 1;  -- 1 = no special offer


**Proposition 8)** Customers who ordered in 2014 but not 2013

- Selects all customers who placed orders in 2014
- Uses an EXCEPT operator to remove any customers who also ordered in 2013
- Returns only new customers who began ordering in 2014.

Why Special: Assists anayltics team in tracking new customers year over year.

In [ ]:
SELECT DISTINCT SalesOrderHeader.CustomerID
FROM Sales.SalesOrderHeader AS SalesOrderHeader
WHERE SalesOrderHeader.OrderDate >= '20140101'
  AND SalesOrderHeader.OrderDate <  '20150101'

EXCEPT

SELECT DISTINCT SalesOrderHeader.CustomerID
FROM Sales.SalesOrderHeader AS SalesOrderHeader
WHERE SalesOrderHeader.OrderDate >= '20130101'
  AND SalesOrderHeader.OrderDate <  '20140101';


**Proposition 9)** Territories where sales were made but no vendors are located

- The first SELECT gets all territories where sales were made(Sales.SalesOrderHeader)
- The second SELECT retrieves all territories where vendors are located by tracing addresses -\> states -\> territories
- The EXCEPT operator removes any territories that appear in both sets

Why Special: helps logistics teams to figure out which region sales are strong but sales presence is missing, helping future vendor expansion.

In [ ]:
SELECT DISTINCT SalesOrderHeader.TerritoryID

FROM Sales.SalesOrderHeader AS SalesOrderHeader
WHERE SalesOrderHeader.TerritoryID IS NOT NULL

EXCEPT

SELECT DISTINCT SalesTerritory.TerritoryID
FROM Purchasing.Vendor AS Vendor
JOIN Person.BusinessEntityAddress AS BusinessEntityAddress
    ON Vendor.BusinessEntityID = BusinessEntityAddress.BusinessEntityID
JOIN Person.Address AS Address
    ON BusinessEntityAddress.AddressID = Address.AddressID
JOIN Person.StateProvince AS StateProvince
    ON Address.StateProvinceID = StateProvince.StateProvinceID
JOIN Sales.SalesTerritory AS SalesTerritory
    ON StateProvince.TerritoryID = SalesTerritory.TerritoryID
WHERE SalesTerritory.TerritoryID IS NOT NULL;


**Proposition 10)** Products available in both 'Red' and 'Black'

- The first query selects all red products
- The second query selects all black products
- The INTERSECT operator returns only the product names that appear in both sets.

Why Special: Useful for inventory and product management to find product lines available in multiple colors.

In [ ]:
SELECT Name
FROM Production.Product
WHERE Color = N'Red'

INTERSECT

SELECT Name
FROM Production.Product
WHERE Color = N'Black';
